In [ ]:
#vocoder

import os

from TTS.trainer import Trainer, TrainingArgs
from TTS.utils.audio import AudioProcessor
from TTS.vocoder.configs import HifiganConfig, MultibandMelganConfig
from TTS.vocoder.datasets.preprocess import load_wav_data
from TTS.vocoder.models.gan import GAN



output_path =os.path.join("./model/vocoder/mbmelgan/")

config = MultibandMelganConfig(
    batch_size=128,
    eval_batch_size=16,
    num_loader_workers=10,
    num_eval_loader_workers=10,
    run_eval=True,
    test_delay_epochs=5,
    epochs=1000,
    seq_len=8192,
    pad_short=2000,
    use_noise_augment=True,
    eval_split_size=10,
    print_step=25,
    print_eval=False,
    mixed_precision=False,
    lr_gen=1e-4,
    lr_disc=1e-4,
    data_path="",
    output_path=output_path,
)


# init audio processor
ap = AudioProcessor(**config.audio.to_dict())


# load training samples
eval_samples1, train_samples1 = load_wav_data("./resample1/kss/wavs", config.eval_split_size)
eval_samples2, train_samples2 = load_wav_data("./resample1/pansori_tedxkr/wavs", config.eval_split_size)
eval_samples3, train_samples3 = load_wav_data("./resample1/zeroth_korean/wavs", config.eval_split_size)
eval_samples, train_samples = [eval_samples1+eval_samples2+eval_samples3, train_samples1+train_samples2+train_samples3]




In [ ]:


# init model
model = GAN(config)

# init the trainer and 🚀
trainer = Trainer(
    TrainingArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
    training_assets={"audio_processor": ap},
)
trainer.fit()